# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Environment
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import json
import joblib

In [4]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [5]:
ws = Workspace.from_config()
experiment = Experiment(workspace=ws, name="Detection")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-137783
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-137783


In [6]:
# TODO: Create compute cluster

# Choose a name for your CPU cluster
cpu_cluster_name = "auto-ml-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)


Creating
Succeeded....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
The original dataset is in Kaggle Datasets. The original data is licensed by Open Database License (ODbL) 1.0.Open Database License (ODbL) 1.0.

This data is about fraud detection in credit card transactions. The data was made by credit cards in September 2013 by European cardholders. The dataset is highly unbalanced, the positive class which depicts fraudulent transactions (frauds) account for 0.17% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we do not have the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with Principal component analysis (PCA), the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise


This project aims to detect potential fraud cases is credit card transactions and the task here is to differentiate between them. My ultimate intent is to tackle this situation by building classification models to classify and distinguish fraud transactions.

In [7]:
path = 'https://media.githubusercontent.com/media/Tekhunt/Creditcard-fraud-dectection/master/fraud-data.csv'
data = TabularDatasetFactory.from_delimited_files(path = path)
data.to_pandas_dataframe().head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## AutoML Settings and Configuration Explained

### n_cross-validation
How many cross validations to perform when user validation data is not specified.

###enable_early_stopping
Whether to enable early termination if the score is not improving in the short term. The default is False but it is set to True here.

### experiment_timeout_minutess
Maximum amount of time in minutes that all iterations combined can take before the experiment terminates. It is set to 15 minutes here.

### verbosity
This is the verbosity level for writing to the log file and it is set to logging.INFO

### training_data
This can be any of these: DataFrame or Dataset or DatasetDefinition or TabularDataset The training data to be used within the experiment. It should contain both training features and a label column (optionally a sample weights column). If training_data is specified, then the label_column_name parameter must also be specified.

### label_column_name
This is the name of the label column. If the input data is from a pandas.DataFrame which doesn't have column names, column indices can be used instead, expressed as integers. Here we have column headers and our arget column is the Class column which we aim to predict in the project.

### max_cores_per_iteration
The maximum number of threads to use for a given training iteration. Acceptable values: Equal to -1, which means to use all the possible cores per iteration per child-run.

### max_concurrent_iterations
Represents the maximum number of iterations that would be executed in parallel. The value used here is 4

### compute_target
The Azure Machine Learning compute target to run the Automated Machine Learning experiment on.

### primary_metric
The metric that Automated Machine Learning will optimize for model selection. Accuracy is the primary_metric here.

### task
The type of task to run. Values the here is 'classification'


In [9]:
# TODO: Put your automl settings here
automl_settings = {
       "n_cross_validations": 3,
       "enable_early_stopping": True,
       "experiment_timeout_minutes" :15,
       "max_concurrent_iterations": 4,
       "max_cores_per_iteration": -1,
       "verbosity": logging.INFO,
   }

# TODO: Puyour experiment name here


automl_config = AutoMLConfig(
    compute_target=cpu_cluster,
    task='classification',
    primary_metric= 'accuracy',
    training_data= data,
    label_column_name= 'Class',
    n_cross_validations=5)

In [10]:
# TODo your experiment name here
#remote_run = experiment.submit(automl_config)

remote_run = experiment.submit(automl_config, show_output=True)
remote_run.wait_for_completion()

Running on remote.
No run_configuration provided, running on auto-ml-cluster with default configuration
Running on remote compute: auto-ml-cluster
Parent Run ID: AutoML_d12b6002-f654-415b-af0c-47d7b7c2d7e8

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

{'runId': 'AutoML_d12b6002-f654-415b-af0c-47d7b7c2d7e8',
 'target': 'auto-ml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T08:11:45.405268Z',
 'endTimeUtc': '2021-02-07T10:12:01.21587Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'auto-ml-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Detection","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-137783","workspace_name":"quick-starts-ws-137783","region":"southcentralus","compute_target":"auto-ml-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [13]:
parameter_values = best_run.get_details()['runDefinition']['arguments']


In [16]:
# Retrieve the best automl model.

print(best_run)
print(fitted_model)

#Register the model
description ='this model predicts whether a transaction is fraudulent or not'
model_name='creditcard-fraud-detection-model'
model_path='./'
tags = None
model = remote_run.register_model(model_name = model_name, description = description , tags = tags)
print(remote_run.model_id)

Run(Experiment: Detection,
Id: AutoML_d12b6002-f654-415b-af0c-47d7b7c2d7e8_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               nthread=None,
                                                                                               objective='reg:logistic',
                                       

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
#Save the best model for the deployement
import os
os.makedirs('./remote_model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./remote_model','best_remote_model.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./remote_model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [18]:
#Register the best model for the deployement

model=best_run.register_model(
            model_name = 'creditcard-fraud-detection-model', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description=''
)

In [19]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [20]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_remote_run.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_remote_run.py',
                                    environment=myenv)

In [21]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [22]:
# deploye the model as a web service
service_name =  'automl-prediction'
service = Model.deploy(workspace=ws, 
                       name= service_name,
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-137783', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-137783'), name=automl-prediction, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [24]:
# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state:
Healthy
Scoring URI:
http://2ad4b6d0-446a-4a12-bdb2-283cb6590e6c.southcentralus.azurecontainer.io/score
Swagger URI:
http://2ad4b6d0-446a-4a12-bdb2-283cb6590e6c.southcentralus.azurecontainer.io/swagger.json


In [25]:
test_data = data.to_pandas_dataframe()

In [27]:
x_data_sample = test_data.sample(3)
y_data_sample = x_data_sample.pop('Class')

# converting samples to json format
json_sample = x_data_sample.to_dict(orient = 'records')#'records'

scoring_sample_data = json.dumps({'data': json_sample})
print(scoring_sample_data)

{"data": [{"Time": 41394.0, "V1": -1.67504125676759, "V2": -0.7767535039861941, "V3": 1.44738070278424, "V4": 0.724940424132646, "V5": -1.12772562786982, "V6": -0.0937594199493375, "V7": -0.3344736073905, "V8": 0.627584708508985, "V9": -1.22116161453843, "V10": 0.213610717781422, "V11": -0.6959419335182829, "V12": -0.0230385571615321, "V13": 0.20000766364138298, "V14": 0.114947034567041, "V15": 0.7044292049147092, "V16": -2.05284686200261, "V17": 0.767940151362606, "V18": 1.2746735364188602, "V19": -0.86119742842079, "V20": 0.25349060417837804, "V21": 0.0954293017583765, "V22": 0.29554501425263896, "V23": 0.39914863104327003, "V24": 0.374379807274368, "V25": -0.15743567758608, "V26": -0.173085543762075, "V27": 0.29189101051942895, "V28": 0.0292837128181044, "Amount": 200.0}, {"Time": 15004.0, "V1": 1.22845486146718, "V2": 0.0494878250902, "V3": 1.02209887089423, "V4": 0.38647107258472896, "V5": -0.9732280332944979, "V6": -1.0678223826533801, "V7": -0.383162369455785, "V8": -0.205406709

In [28]:

output = service.run(scoring_sample_data)

In [29]:
print(output)

{"result": [0, 0, 0]}


In [30]:
print(y_data_sample)

43154    0
9996     0
48461    0
Name: Class, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [31]:
print(service.get_logs())

2021-02-07T10:32:28,623846400+00:00 - iot-server/run 
2021-02-07T10:32:28,634768500+00:00 - gunicorn/run 
2021-02-07T10:32:28,636334300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-07T10:32:28,656152200+00:00 - rsyslog/run 
rsyslogd

In [32]:

service.delete()
aml_compute.delete()

NameError: name 'aml_compute' is not defined